In [ ]:
import numpy as np
import cv2 as cv
import os
# from sklearn.model_selection import train_test_spli
from sklearn.neighbors import KNeighborsClassifier
######### KNN CODE ################
def distance(v1, v2):
    return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k= 5):
    dist = []
    for i in range(train.shape[0]):
        # Get the vectore and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
        
    # sort based on distance and get top k
    dk = sorted(dist, key = lambda x:x[0])[:k]
    # retrieve only the labels
    labels = np.array(dk)[:, -1]
    
    # Get frequences of each label
    output = np.unique(labels, return_counts= True)
    # find max frequency and corresponding label
    index = np.argmax(output[1])
    
    return output[0][index]


# def knn(train, test, k = 5):
#     knn = KNeighborsClassifier()
#     knn.fit(train, test)
#     result = knn.predict(test)
#     return result
    

############## END ##################

########## Capture Face And Store in the data File #######

cap = cv.VideoCapture(0)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

dataset_path = "./face_dataset/"

face_data = []
labels = []
class_id = 0
names = {}

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)
        
        target = class_id * np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)
        
face_dataset = np.concatenate(face_data, axis = 0)
face_labels = np.concatenate(labels, axis = 0).reshape((-1, 1))
print(face_labels.shape)
print(face_dataset.shape)

trainset = np.concatenate((face_dataset, face_labels), axis = 1)
print(trainset.shape)

font = cv.FONT_HERSHEY_SIMPLEX

while True:
    ret, frame = cap.read()
    if ret == False:
        continue
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for face in faces:
        x, y, w, h = face
        
        # get the face ROI
        offset = 5
        face_section = frame[y-offset:y+h+offset, x-offset: x+w+offset]
        face_section = cv.resize(face_section, (100, 100))
        
        out = knn(trainset, face_section.flatten())
        
        # draw the rectangle in the original image
        cv.putText(frame, names[int(out)], (x, y-10), font, 1, (255, 0, 0), 2, cv.LINE_AA)
        cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    cv.imshow('Faces', frame)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()
        
        
        
        

(72, 1)
(72, 30000)
(72, 30001)
